In [95]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [96]:
class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):


        """
            Some initializations, if neccesary

            attributes:
                        alpha: Learning Rate, default 1e-10
                        num_iter: Number of Iterations to update coefficient with training data
                        early_stop: Constant control early_stop.
                        intercept: Bool, If we are going to fit a intercept, default True.
                        init_weight: Matrix (n x 1), input init_weight for testing.


            TODO: 1. Initialize all variables needed.
        """
        self.alpha = alpha
        self.num_iter = num_iter
        self.early_stop = early_stop
        self.intercept = intercept
        self.init_weight = init_weight
        self.model_name = 'Linear Regression'
        self.coef = None

    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and perform gradient descent.

            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                Y_train: Matrix or 2-D array. Input target value.


                TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
                      3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
                      4. Call the gradient_descent function to train.
        """

        self.X = np.array(X_train)
        self.y = np.array(y_train)

        if self.intercept:
            self.X = np.hstack((np.ones((self.X.shape[0], 1)), self.X))

        num_cols = self.X.shape[1]
        self.coef = np.random.uniform(-1, 1, size=(num_cols, 1))

        self.gradient_descent()

        return self
    

    def gradient(self):
        """
            Helper function to calculate the gradient respect to coefficient.

            TODO: 5. Think about the matrix format of the gradient of the loss function.
        """

        y_pred = np.dot(self.X, self.coef)
        error = y_pred - self.y
        return np.dot(self.X.T, error) / len(self.y)

    def gradient_descent(self):
        """
        Training function

        TODO:
        6. Calculate the loss with current coefficients.
        7. Update the temp_coef with learning rate and gradient.
        8. Calculate the loss with temp_coef.
        9. Implement the self-adaptive learning rate.
            a. If current error is less than previous error, increase learning rate by a factor of 1.3.
              And update coef with temp_coef.
            b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
              Don't update coef.
        10. Add the loss to the loss list we create.
        """

        self.loss = []

        for _ in range(self.num_iter):
            
            y_pred = np.dot(self.X, self.coef)
            current_error = np.mean((y_pred - self.y) ** 2)
            self.loss.append(current_error)
            
            gradient = self.gradient()
            temp_coef = self.coef - self.alpha * gradient

            y_pred_temp = np.dot(self.X, temp_coef)
            temp_error = np.mean((y_pred_temp - self.y) ** 2)

            if temp_error < self.early_stop or (self.loss and abs(temp_error - self.loss[-1]) < self.early_stop):
                break

            if temp_error < np.mean((np.dot(self.X, self.coef) - self.y) ** 2):
                self.coef = temp_coef
                self.alpha *= 1.3
            else:
                self.alpha *= 0.9

            self.loss.append(temp_error)
            
            if abs(temp_error - current_error) < self.early_stop or abs(temp_error / current_error - 1) < self.early_stop:
                break

    def ind_predict(self, x: list):
        """
        Predict the value based on its feature vector x.

        Parameter:
            x: Matrix, array or list. Input feature point.

        Return:
            result: prediction of given data point
        """

        """
        TODO:
        11. Implement the prediction function
        """

        x = np.array(x)
        prediction = np.dot(x, self.coef)
        return prediction.item()

    def predict(self, X):
        X = np.array(X)

        if self.intercept:
            X = np.hstack((np.ones((X.shape[0], 1)), X))

        predictions = np.dot(X, self.coef)
        return predictions.flatten()

In [97]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [98]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() +  20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [99]:
clf = Linear_Regression(alpha = 1, num_iter = 1000)
clf.fit(X,y)

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30].
#### It maybe very slow update. Feel free to stop.

In [100]:
clf.predict(X)

array([9.68219144e-01, 1.25036346e+00, 1.53250778e+00, ...,
       4.41864550e+04, 4.44103701e+04, 4.46342851e+04])

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

In [101]:
# Calculate mean and standard deviation of the feature matrix X
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)

# Normalize the feature matrix X
X_normalized = (X - X_mean) / X_std

# Initialize a Linear Regression model with specified hyperparameters
clf_normalized = Linear_Regression(alpha=1, num_iter=1000)

# Fit the model to the normalized feature matrix X_normalized and target values y
clf_normalized.fit(X_normalized, y)

# Normalize the test data using the same mean and standard deviation as the training data
X_test_normalized = (X - X_mean) / X_std

# Obtain predictions from the fitted model using the normalized test data
predictions_normalized = clf_normalized.predict(X_test_normalized)

# Print the predictions obtained from the model trained on normalized data
print("Predictions (normalized):", predictions_normalized)

Predictions (normalized): [   50.   200.   350. ... 29600. 29750. 29900.]


##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report.

In [102]:
from sklearn.linear_model import LinearRegression

In [103]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality

In [104]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

800.6676988774337

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [105]:
clf = Linear_Regression(alpha = 1, num_iter = 1)
clf.fit(X,y)

In [106]:
sum((clf.predict(X) - y)**2)

247899.12638757518